In [1]:
# First: load the model with vllm:

# vllm serve unsloth/gemma-3-1b-it --port 8008 --max-model-len 2500 --served-model-name unsloth/gemma-3-1b-it --gpu-memory-utilization 0.8 --dtype bfloat16 --api-key xxx

In [2]:
import subprocess
from pathlib import Path
from collections import Counter

In [3]:
# Example 1: Direct prompting mode. Returns a single unslopped output.

if False:
    cmd = [
        'python3', 'main.py',
        #'--api-base-url', 'http://localhost:8008/v1',
        '--api-key', 'xxx',
        '--model-name', 'Qwen/Qwen3-4B',
        '--chat-template-model-id', 'Qwen/Qwen3-4B',
        '--logging-level', 'INFO',
        '--slop-phrases-file', 'banlists/slop_phrases.json',
        '--top-n-slop-phrases', '500',
        '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
        '--max-new-tokens', '500',
        '--prompt', 'Write a story about a princess. '
    ]
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=True)

In [ ]:
# Example 2: Generate an unslopped dataset using prompts from a huggingface sharegpt dataset

THREADS  = 60
MAX_PROMPTS = 60

OUT_JSONL     = 'creative_writing_generations.jsonl'
HUMAN_PROFILE = 'data/human_writing_profile.json'


cmd = [
    'python3', 'main.py',
    #'--api-base-url', 'http://localhost:8008/v1',
    '--api-key', 'xxx',
    '--model-name', 'unsloth/gemma-3-4b-it',
    '--output-jsonl', OUT_JSONL,
    '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT',
    '--hf-dataset-split', 'train',
    '--threads', str(THREADS),
    '--max-prompts', str(MAX_PROMPTS),
    '--logging-level', 'DEBUG',
    '--slop-phrases-file', 'banlists/slop_phrases.json',
    '--top-n-slop-phrases', '500',
    '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
    '--max-new-tokens', '2000',
    '--chunk-size', '20',
    '--max-new-tokens', '300',
    '--chat-template-model-id', "unsloth/gemma-3-4b-it",
    '--tdpo-pairs-jsonl', 'tdpo-pairs.jsonl'
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


Running: python3 main.py --api-key xxx --model-name unsloth/gemma-3-4b-it --output-jsonl creative_writing_generations.jsonl --input-hf-dataset Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT --hf-dataset-split train --threads 60 --max-prompts 1 --logging-level DEBUG --slop-phrases-file banlists/slop_phrases.json --top-n-slop-phrases 500 --max-new-tokens 2000 --chunk-size 20 --chat-template-model-id unsloth/gemma-3-4b-it --tdpo-pairs-jsonl tdpo-pairs.jsonl


2025-05-13 10:59:16,317 [INFO ] [root                ]: Full DEBUG logging enabled. Effective script level: DEBUG
2025-05-13 10:59:16,317 [DEBUG] [root                ]: Running in batch data generation mode.
2025-05-13 10:59:16,317 [INFO ] [root                ]: tdpo-pair capture enabled → tdpo-pairs.jsonl
2025-05-13 10:59:16,317 [DEBUG] [root                ]: Loading prompts from HF dataset: Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT
2025-05-13 10:59:16,557 [DEBUG] [urllib3.connectionpo]: https://huggingface.co:443 "HEAD /datasets/Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT/resolve/main/README.md HTTP/1.1" 200 0
2025-05-13 10:59:17,485 [DEBUG] [urllib3.connectionpo]: https://huggingface.co:443 "HEAD /datasets/Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT/resolve/ff1964bf78f457f33a87020a3ba7ede40ffeff67/Reddit-SFW-Writing_Prompts_ShareGPT.py HTTP/1.1" 404 0
2025-05-13 10:59:17,486 [DEBUG] [urllib3.connectionpo]: Starting new HTTPS connection (1): s3.amazonaws.com:443
2025-05-13

Extracting HF prompts:   0%|          | 0/177477 [00:00<?, ?prompt/s]
Preparing to process 1 new prompts in this run.
Batch Generating:   0%|          | 0/1 [00:00<?, ?prompt/s]

2025-05-13 10:59:21,645 [DEBUG] [urllib3.connectionpo]: https://ncfi2yswjottwq-8888.proxy.runpod.net:443 "POST /v1/completions HTTP/1.1" 200 None
2025-05-13 10:59:21,799 [DEBUG] [api_client.api_clien]: Raw response: {"id": "cmpl-8284f242d0dd405292f54145aa110d72", "object": "text_completion", "created": 1747097966, "model": "unsloth/gemma-3-4b-it", "choices": [{"index": 0, "text": "The rain in Georgia was a greasy, insistent thing, clinging to everything with the stubbornness of regret", "logprobs": {"text_offset": [0, 3, 8, 11, 19, 23, 25, 32, 33, 43, 49, 50, 59, 62, 73, 78, 82, 91, 95, 98], "token_logprobs": [-1.311301275563892e-06, -0.11339403688907623, -0.08055462688207626, -0.014241487719118595, -1.6848098039627075, -0.403470903635025, -0.5678627490997314, -0.013264154084026814, -0.0475982241332531, -0.02536853961646557, -0.0021594080608338118, -0.19066542387008667, -3.433168603805825e-05, -0.6955522894859314, -2.594970703125, -0.28212910890579224, -1.8402045965194702, -0.094360195

Batch Generating:   0%|          | 0/1 [00:02<?, ?prompt/s, 70.2 tok/s]

...

2025-05-13 11:00:59,475 [DEBUG] [urllib3.connectionpo]: https://ncfi2yswjottwq-8888.proxy.runpod.net:443 "POST /v1/completions HTTP/1.1" 200 None
2025-05-13 11:00:59,538 [DEBUG] [api_client.api_clien]: Raw response: {"id": "cmpl-5e5e5ae1636b4ba98c4452081ee771ab", "object": "text_completion", "created": 1747098064, "model": "unsloth/gemma-3-4b-it", "choices": [{"index": 0, "text": ".  He was the best that\u2019s ever been? Maybe. Or maybe he was just the", "logprobs": {"text_offset": [0, 1, 3, 5, 9, 13, 18, 23, 24, 25, 30, 35, 36, 42, 43, 46, 52, 55, 59, 64], "token_logprobs": [-0.04900036007165909, -0.7569708824157715, -0.6274473667144775, -1.1957111358642578, -0.6549874544143677, -0.004187862388789654, -0.04244169965386391, -0.4545176029205322, -0.026302536949515343, -0.0007727735792286694, -0.0004481264913920313, -0.5514078140258789, -0.763495147228241, -0.03408034145832062, -1.6943533420562744, -0.0017171651124954224, -0.9044760465621948, -0.013038027100265026, -0.08120221644639969,

Batch Generating:   0%|          | 0/1 [01:39<?, ?prompt/s, 18.0 tok/s]

2025-05-13 11:01:00,089 [DEBUG] [urllib3.connectionpo]: https://ncfi2yswjottwq-8888.proxy.runpod.net:443 "POST /v1/completions HTTP/1.1" 200 None
2025-05-13 11:01:00,148 [DEBUG] [api_client.api_clien]: Raw response: {"id": "cmpl-4bed0b1c7bf9498cb39ee721ebd60b66", "object": "text_completion", "created": 1747098065, "model": "unsloth/gemma-3-4b-it", "choices": [{"index": 0, "text": " first to realize he had a long, very complicated battle ahead of him. \n", "logprobs": {"text_offset": [0, 6, 9, 17, 20, 24, 26, 31, 32, 37, 49, 56, 62, 65, 69, 70, 71, 72], "token_logprobs": [-0.6203574538230896, -0.15540064871311188, -1.1121540069580078, -1.1823041439056396, -0.301325261592865, -0.005291503854095936, -1.5698975324630737, -0.052742090076208115, -1.7516326904296875, -0.36661726236343384, -0.09811710566282272, -0.005525553598999977, -0.011086059734225273, -0.001035987981595099, -0.2718963027000427, -1.4307894706726074, -0.09351818263530731, 0.0], "tokens": ["\u2581first", "\u2581to", "\u2581r

Batch Generating: 100%|██████████| 1/1 [01:40<00:00, 100.04s/prompt, 18.1 tok/s]
Finished processing 1 prompts in this run in 100.04s.
Overall average throughput for this run: 17.78 tok/s.
Results appended to creative_writing_generations.jsonl


CompletedProcess(args=['python3', 'main.py', '--api-key', 'xxx', '--model-name', 'unsloth/gemma-3-4b-it', '--output-jsonl', 'creative_writing_generations.jsonl', '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT', '--hf-dataset-split', 'train', '--threads', '60', '--max-prompts', '1', '--logging-level', 'DEBUG', '--slop-phrases-file', 'banlists/slop_phrases.json', '--top-n-slop-phrases', '500', '--max-new-tokens', '2000', '--chunk-size', '20', '--chat-template-model-id', 'unsloth/gemma-3-4b-it', '--tdpo-pairs-jsonl', 'tdpo-pairs.jsonl'], returncode=0)